Thesis Data Preprocessing

I will clean and processing 4 raw datasets and then merge them into two datasets as shown below:

City Dataset
- 2013 city GHG emissions per capita
- 2017 Mayoral support (signature) for Paris climate agreement
- 2000-2011 City Partisan index
- Mayoral political party (for a mayor that had the opportunity to sign support for paris)

Start with the City Dataset

Step 1: 2013 GHG emissions per capita

In [1]:
from os import path
import requests
from bs4 import BeautifulSoup as Soup
from urllib.request import Request, urlopen
from pandas import DataFrame
import pandas as pd
import numpy as np
import math
DATA_DIR = '/Users/joeskvarna/Desktop/Thesis/Data'
#pd.set_option('display.max_rows', None)
pd.set_option('max_rows', 5)

In [2]:
city = pd.read_csv(path.join(DATA_DIR, 'city_ghg_source.txt'), sep =  '\t')
city.head(10)

,Urban Cluster,Country,Population,Footprint (Mt CO2),1StdDev,Footprint/Cap (t CO2/cap),ClusterID,Global ranking
0,Hong Kong SAR,China,6029000,NaN,37.8,34.6 &plusmn;6.3,13847,1
1,Mohammed Bin Zayed City,UAE,188000,NaN,5.3,32.9 &plusmn;27.9,5292,2
...,...,...,...,...,...,...,...,...
8,Chifeng,China,677000,NaN,11.0,28.0 &plusmn;16.2,11095,9
9,Al-Jahrah,Kuwait,178000,NaN,3.3,27.2 &plusmn;18.4,5116,10


In [3]:
city_us = city.loc[city['Country'] == 'USA']
city_us = city_us.reset_index(drop=True)
city_us

,Urban Cluster,Country,Population,Footprint (Mt CO2),1StdDev,Footprint/Cap (t CO2/cap),ClusterID,Global ranking
0,New Orleans,USA,596000,NaN,7.8,26.1 &plusmn;13.0,373,12
1,Detroit,USA,2770000,NaN,15.4,25.5 &plusmn;5.6,16079,13
...,...,...,...,...,...,...,...,...
107,"Unknown city at lat/lon 33.5, -117.2",USA,193000,NaN,1.3,9.6 &plusmn;6.9,39,425
108,"Unknown city at lat/lon 27.3, -80.4",USA,170000,NaN,0.8,9.2 &plusmn;5.0,550,471


In [4]:
play_df = city_us
city_us["Urban Cluster"] = city_us["Urban Cluster"].str.replace(r',[^,]*$', '')
city_us

<ipython-input-4-26db4f1c9bd4>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  city_us["Urban Cluster"] = city_us["Urban Cluster"].str.replace(r',[^,]*$', '')


,Urban Cluster,Country,Population,Footprint (Mt CO2),1StdDev,Footprint/Cap (t CO2/cap),ClusterID,Global ranking
0,New Orleans,USA,596000,NaN,7.8,26.1 &plusmn;13.0,373,12
1,Detroit,USA,2770000,NaN,15.4,25.5 &plusmn;5.6,16079,13
...,...,...,...,...,...,...,...,...
107,Unknown city at lat/lon 33.5,USA,193000,NaN,1.3,9.6 &plusmn;6.9,39,425
108,Unknown city at lat/lon 27.3,USA,170000,NaN,0.8,9.2 &plusmn;5.0,550,471


Step 2: 2015 Mayoral support for Paris Climate Agreement

In [5]:
#scrape the data from website
url = 'https://climatemayors.org/actions-paris-climate-agreement'
def scraper(url):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    page_soup = Soup(webpage, 'html.parser')
    return page_soup
page_soup = scraper(url)

In [6]:
#return a list of p tags
p = page_soup.find_all('p')
p[:5]

[<p>STATEMENT FROM THE CLIMATE MAYORS IN RESPONSE TO PRESIDENT TRUMP’S WITHDRAWAL FROM THE PARIS CLIMATE AGREEMENT</p>,
 <p>The President’s denial of global warming is getting a cold reception from America’s cities.</p>,
 <p>As 466 US Mayors representing 74 million Americans, we will adopt, honor, and uphold the commitments to the goals enshrined in the Paris Agreement. We will intensify efforts to meet each of our cities’ current climate goals, push for new action to meet the 1.5 degrees Celsius target, and work together to create a 21st century clean energy economy.</p>,
 <p>We will continue to lead. We are increasing investments in renewable energy and energy efficiency. We will buy and create more demand for electric cars and trucks. We will increase our efforts to cut greenhouse gas emissions, create a clean energy economy, and stand for environmental justice. And if the President wants to break the promises made to our allies enshrined in the historic Paris Agreement, we’ll build

In [7]:
#split p tags by line
split_list = [str(x).splitlines() for x in p[6:-2]]
split_list[:5]

[['<p>Mayor Trish Herrera Spencer<br/>', 'City of Alameda, CA</p>'],
 ['<p>Mayor Peggy McQuaid<br/>', 'City of Albany, CA</p>'],
 ['<p>Mayor Sharon Konopa<br/>', 'City of Albany, OR</p>'],
 ['<p>Mayor Kathy Sheehan<br/>', 'City of Albany, NY</p>'],
 ['<p>Mayor Tim Keller<br/>', 'City of Albuquerque, NM</p>']]

In [8]:
#convert n x 2 list into a 2 column dataframe
mayor_paris = pd.DataFrame(split_list)
mayor_paris

,0,1
0,<p>Mayor Trish Herrera Spencer<br/>,"City of Alameda, CA</p>"
1,<p>Mayor Peggy McQuaid<br/>,"City of Albany, CA</p>"
...,...,...
464,<p>Mayor Jamael Tito Brown<br/>,"City of Youngstown, OH</p>"
465,<p>Mayor Lois Richardson<br/>,"City of Ypsilanti, MI</p>"


In [9]:
#rename columns
mayor_paris = mayor_paris.rename({0:'mayor_name', 1:'mayor_city'}, axis = 1)
mayor_paris

,mayor_name,mayor_city
0,<p>Mayor Trish Herrera Spencer<br/>,"City of Alameda, CA</p>"
1,<p>Mayor Peggy McQuaid<br/>,"City of Albany, CA</p>"
...,...,...
464,<p>Mayor Jamael Tito Brown<br/>,"City of Youngstown, OH</p>"
465,<p>Mayor Lois Richardson<br/>,"City of Ypsilanti, MI</p>"


In [10]:
#clean the 2 columns
#there are some dups due to same city name in different states, we'll worry about this later
if len(mayor_paris['mayor_city'][0]) >= 12:
    mayor_paris['mayor_name'] = mayor_paris['mayor_name'].str[9:-5]
    mayor_paris['mayor_city'] = mayor_paris['mayor_city'].str[8:-4]
    
if mayor_paris.at[292, 'mayor_city'] != 'New York, NY':
    mayor_paris.at[292, 'mayor_city'] = 'New York, NY'
    
if mayor_paris.at[434, 'mayor_city'] != 'Washington D.C., DC':
    mayor_paris.at[434, 'mayor_city'] = 'Washington D.C., DC'

if mayor_paris.at[396, 'mayor_city'] != 'St. Louis, MO':
    mayor_paris.at[396, 'mayor_city'] = 'St. Louis, MO'

mayor_paris

,mayor_name,mayor_city
0,Trish Herrera Spencer,"Alameda, CA"
1,Peggy McQuaid,"Albany, CA"
...,...,...
464,Jamael Tito Brown,"Youngstown, OH"
465,Lois Richardson,"Ypsilanti, MI"


Step 3: City Partisan Index

In [11]:
#read the csv file from a study already done
city_partisan = pd.read_csv(path.join(DATA_DIR, 'city_partisan_index.csv'))
city_partisan

,City,Conservatism
0,"Mesa, AZ",0.41
1,"Oklahoma City, OK",0.26
...,...,...
65,"Washington, DC",-0.93
66,"San Francisco, CA",-1.00


Step 4: Set up to merge these 3 into the format we want

In [12]:
city_us.head()

,Urban Cluster,Country,Population,Footprint (Mt CO2),1StdDev,Footprint/Cap (t CO2/cap),ClusterID,Global ranking
0,New Orleans,USA,596000,NaN,7.8,26.1 &plusmn;13.0,373,12
1,Detroit,USA,2770000,NaN,15.4,25.5 &plusmn;5.6,16079,13
2,Cleveland,USA,973000,NaN,8.8,24.7 &plusmn;9.0,726,17
3,St. Louis,USA,1177000,NaN,9.6,24.1 &plusmn;8.2,448,19
4,Pittsburgh,USA,602000,NaN,6.3,24.0 &plusmn;10.4,753,21


In [13]:
mayor_paris.head()

,mayor_name,mayor_city
0,Trish Herrera Spencer,"Alameda, CA"
1,Peggy McQuaid,"Albany, CA"
2,Sharon Konopa,"Albany, OR"
3,Kathy Sheehan,"Albany, NY"
4,Tim Keller,"Albuquerque, NM"


In [14]:
city_partisan.head()

,City,Conservatism
0,"Mesa, AZ",0.41
1,"Oklahoma City, OK",0.26
2,"Virginia Beach, VA",0.25
3,"Colorado Springs, CO",0.23
4,"Jacksonville, FL",0.18


In [15]:
#Combine city_us and city_partisan first
#I will remove the state part of city_partisan
if len(city_partisan['City'][0]) >= 5:
    city_partisan['City'] = city_partisan['City'].str[:-4]
city_partisan

,City,Conservatism
0,Mesa,0.41
1,Oklahoma City,0.26
...,...,...
65,Washington,-0.93
66,San Francisco,-1.00


In [16]:
#merge both on their respective city column, 'Urban Cluster' and 'City'
comb_1 = pd.merge(city_us, city_partisan, how='left', left_on = 'Urban Cluster', right_on = 'City')
comb_1

,Urban Cluster,Country,Population,Footprint (Mt CO2),1StdDev,Footprint/Cap (t CO2/cap),ClusterID,Global ranking,City,Conservatism
0,New Orleans,USA,596000,NaN,7.8,26.1 &plusmn;13.0,373,12,New Orleans,-0.51
1,Detroit,USA,2770000,NaN,15.4,25.5 &plusmn;5.6,16079,13,Detroit,-0.73
...,...,...,...,...,...,...,...,...,...,...
107,Unknown city at lat/lon 33.5,USA,193000,NaN,1.3,9.6 &plusmn;6.9,39,425,NaN,NaN
108,Unknown city at lat/lon 27.3,USA,170000,NaN,0.8,9.2 &plusmn;5.0,550,471,NaN,NaN


In [17]:
#Create a list of the Urban Cluster and City column from comb_1
#and a list of the city column from the city_partisan table. And check length of all.
comb_1_city = (comb_1['City'].dropna()).tolist()
comb_1_urban = (comb_1['Urban Cluster'].dropna()).tolist()
city_partisan_city = (city_partisan['City'].dropna()).tolist()
print(len(comb_1_city))
print(len(comb_1_urban))
print(len(city_partisan_city))

50
109
67


In [18]:
#I will see which cities I lost in the merge from city_partisan. It should be a list of 17 cities.
city_partisan_lost = []
for city in city_partisan_city:
    if city not in comb_1_city:
        city_partisan_lost.append(city)

print(city_partisan_lost)
print(len(city_partisan_lost))

['Mesa', 'Virginia Beach', 'Arlington', 'Anaheim', 'Aurora', 'Anchorage', 'Fort Worth', 'Lexington', 'Riverside', 'Santa Ana', 'Long Beach', 'Honolulu', 'Newark', 'Pittrburgh', 'St. Paul', 'Washington', 'San Francisco']
17


In [19]:
#17 as we would expect (67-50)
#We will take note of Pittsburgh and Washington as they are spelled differently. The ghg dataset doesn't have San Francisco
city_partisan_lost

['Mesa',
 'Virginia Beach',
 'Arlington',
 'Anaheim',
 'Aurora',
 'Anchorage',
 'Fort Worth',
 'Lexington',
 'Riverside',
 'Santa Ana',
 'Long Beach',
 'Honolulu',
 'Newark',
 'Pittrburgh',
 'St. Paul',
 'Washington',
 'San Francisco']

In [20]:
comb_1

,Urban Cluster,Country,Population,Footprint (Mt CO2),1StdDev,Footprint/Cap (t CO2/cap),ClusterID,Global ranking,City,Conservatism
0,New Orleans,USA,596000,NaN,7.8,26.1 &plusmn;13.0,373,12,New Orleans,-0.51
1,Detroit,USA,2770000,NaN,15.4,25.5 &plusmn;5.6,16079,13,Detroit,-0.73
...,...,...,...,...,...,...,...,...,...,...
107,Unknown city at lat/lon 33.5,USA,193000,NaN,1.3,9.6 &plusmn;6.9,39,425,NaN,NaN
108,Unknown city at lat/lon 27.3,USA,170000,NaN,0.8,9.2 &plusmn;5.0,550,471,NaN,NaN


Step 5: Put mayor support for climate change in as boolean

In [21]:
#here are all the mayors and cities that signed support for the Paris Climate accord
mayor_paris

,mayor_name,mayor_city
0,Trish Herrera Spencer,"Alameda, CA"
1,Peggy McQuaid,"Albany, CA"
...,...,...
464,Jamael Tito Brown,"Youngstown, OH"
465,Lois Richardson,"Ypsilanti, MI"


In [22]:
#we can add the mayor_city column to comb_1 to create the boolean column. I will also leave the mayor_name column
if len(mayor_paris['mayor_city'][0]) >= 8:
    mayor_paris['mayor_city'] = mayor_paris['mayor_city'].str[:-4]
mayor_paris

,mayor_name,mayor_city
0,Trish Herrera Spencer,Alameda
1,Peggy McQuaid,Albany
...,...,...
464,Jamael Tito Brown,Youngstown
465,Lois Richardson,Ypsilanti


In [23]:
#combine mayor_paris and comb_1 on mayor_city and Urban Cluster
city_dataset = pd.merge(comb_1, mayor_paris, how='left', left_on = 'Urban Cluster', right_on = 'mayor_city')
city_dataset

,Urban Cluster,Country,Population,Footprint (Mt CO2),1StdDev,Footprint/Cap (t CO2/cap),ClusterID,Global ranking,City,Conservatism,mayor_name,mayor_city
0,New Orleans,USA,596000,NaN,7.8,26.1 &plusmn;13.0,373,12,New Orleans,-0.51,LaToya Cantrell,New Orleans
1,Detroit,USA,2770000,NaN,15.4,25.5 &plusmn;5.6,16079,13,Detroit,-0.73,Mike Duggan,Detroit
...,...,...,...,...,...,...,...,...,...,...,...,...
114,Unknown city at lat/lon 33.5,USA,193000,NaN,1.3,9.6 &plusmn;6.9,39,425,NaN,NaN,NaN,NaN
115,Unknown city at lat/lon 27.3,USA,170000,NaN,0.8,9.2 &plusmn;5.0,550,471,NaN,NaN,NaN,NaN


In [24]:
#create boolean column on whether the cities mayor signed support
city_dataset['supports_paris'] = city_dataset['Urban Cluster'] == city_dataset['mayor_city']
city_dataset

,Urban Cluster,Country,Population,Footprint (Mt CO2),1StdDev,Footprint/Cap (t CO2/cap),ClusterID,Global ranking,City,Conservatism,mayor_name,mayor_city,supports_paris
0,New Orleans,USA,596000,NaN,7.8,26.1 &plusmn;13.0,373,12,New Orleans,-0.51,LaToya Cantrell,New Orleans,True
1,Detroit,USA,2770000,NaN,15.4,25.5 &plusmn;5.6,16079,13,Detroit,-0.73,Mike Duggan,Detroit,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,Unknown city at lat/lon 33.5,USA,193000,NaN,1.3,9.6 &plusmn;6.9,39,425,NaN,NaN,NaN,NaN,False
115,Unknown city at lat/lon 27.3,USA,170000,NaN,0.8,9.2 &plusmn;5.0,550,471,NaN,NaN,NaN,NaN,False


In [25]:
#rename and cleanup dataset
if 'Urban Cluster' in city_dataset.columns:
    city_dataset = city_dataset[['Urban Cluster', 'mayor_name','Footprint/Cap (t CO2/cap)', 'Conservatism', 'supports_paris']]
    city_dataset = city_dataset.rename(columns = {'Urban Cluster': 'city', 'Footprint/Cap (t CO2/cap)':'CO2_per_capita', 'Conservatism':'conservatism' })
city_dataset

,city,mayor_name,CO2_per_capita,conservatism,supports_paris
0,New Orleans,LaToya Cantrell,26.1 &plusmn;13.0,-0.51,True
1,Detroit,Mike Duggan,25.5 &plusmn;5.6,-0.73,True
...,...,...,...,...,...
114,Unknown city at lat/lon 33.5,NaN,9.6 &plusmn;6.9,NaN,False
115,Unknown city at lat/lon 27.3,NaN,9.2 &plusmn;5.0,NaN,False


Step X:

In [26]:
#now that the data is cleaner I want to bring back the city_partisan_lost list
city_partisan_lost

['Mesa',
 'Virginia Beach',
 'Arlington',
 'Anaheim',
 'Aurora',
 'Anchorage',
 'Fort Worth',
 'Lexington',
 'Riverside',
 'Santa Ana',
 'Long Beach',
 'Honolulu',
 'Newark',
 'Pittrburgh',
 'St. Paul',
 'Washington',
 'San Francisco']

In [27]:
#In this we see Pittsburgh is spelled wrong and Washington for Washington D.C. is spelled differently
print(city_partisan_lost[-2], city_partisan_lost[-4])
print(city_dataset.iloc[4,0], city_dataset.iloc[40,0])

Washington Pittrburgh
Pittsburgh Washington D.C.


In [28]:
city_partisan

,City,Conservatism
0,Mesa,0.41
1,Oklahoma City,0.26
...,...,...
65,Washington,-0.93
66,San Francisco,-1.00


In [29]:
#This is why the two cities didn't merge!
#I will create a sub datadrame with just these values from city_partisan and merge with city_datset to get the conservatism score in the city_dataset
sub_city_partisan = city_partisan.loc[[48, 65]]
sub_city_partisan = sub_city_partisan.rename(columns = {'City':'city', 'Conservatism': 'conservatism'})
sub_city_partisan

,city,conservatism
48,Pittrburgh,-0.49
65,Washington,-0.93


In [30]:
sub_city_partisan['city'].replace({'Pittrburgh': 'Pittsburgh', 'Washington': 'Washington D.C.'}, inplace = True)
sub_city_partisan

,city,conservatism
48,Pittsburgh,-0.49
65,Washington D.C.,-0.93


In [31]:
city_dataset = pd.merge(city_dataset, sub_city_partisan, how='left', left_on = 'city', right_on = 'city')
city_dataset

,city,mayor_name,CO2_per_capita,conservatism_x,supports_paris,conservatism_y
0,New Orleans,LaToya Cantrell,26.1 &plusmn;13.0,-0.51,True,NaN
1,Detroit,Mike Duggan,25.5 &plusmn;5.6,-0.73,True,NaN
...,...,...,...,...,...,...
114,Unknown city at lat/lon 33.5,NaN,9.6 &plusmn;6.9,NaN,False,NaN
115,Unknown city at lat/lon 27.3,NaN,9.2 &plusmn;5.0,NaN,False,NaN


In [32]:
city_dataset

,city,mayor_name,CO2_per_capita,conservatism_x,supports_paris,conservatism_y
0,New Orleans,LaToya Cantrell,26.1 &plusmn;13.0,-0.51,True,NaN
1,Detroit,Mike Duggan,25.5 &plusmn;5.6,-0.73,True,NaN
...,...,...,...,...,...,...
114,Unknown city at lat/lon 33.5,NaN,9.6 &plusmn;6.9,NaN,False,NaN
115,Unknown city at lat/lon 27.3,NaN,9.2 &plusmn;5.0,NaN,False,NaN


In [33]:
if 'conservatism_y' in city_dataset.columns:
    city_dataset.conservatism_x.fillna(city_dataset.conservatism_y, inplace = True)
    city_dataset.drop(['conservatism_y'], axis = 1, inplace = True)
city_dataset = city_dataset.rename(columns={'conservatism_x': 'conservatism'})
city_dataset

,city,mayor_name,CO2_per_capita,conservatism,supports_paris
0,New Orleans,LaToya Cantrell,26.1 &plusmn;13.0,-0.51,True
1,Detroit,Mike Duggan,25.5 &plusmn;5.6,-0.73,True
...,...,...,...,...,...
114,Unknown city at lat/lon 33.5,NaN,9.6 &plusmn;6.9,NaN,False
115,Unknown city at lat/lon 27.3,NaN,9.2 &plusmn;5.0,NaN,False


In [34]:
mayor_paris

,mayor_name,mayor_city
0,Trish Herrera Spencer,Alameda
1,Peggy McQuaid,Albany
...,...,...
464,Jamael Tito Brown,Youngstown
465,Lois Richardson,Ypsilanti


In [35]:
city_partisan_lost

['Mesa',
 'Virginia Beach',
 'Arlington',
 'Anaheim',
 'Aurora',
 'Anchorage',
 'Fort Worth',
 'Lexington',
 'Riverside',
 'Santa Ana',
 'Long Beach',
 'Honolulu',
 'Newark',
 'Pittrburgh',
 'St. Paul',
 'Washington',
 'San Francisco']

In [36]:
city_dataset.to_csv('city_dataset_draft1')
#lost city partisan and mayor paris with unaligned names of city_us 'Urban Cluster column'. Look to take away - and , that would be great
#some issues with mayor_paris cities not aligning with cities, go back and look at normalizing name of city_us in Urban Cluster

In [37]:
DATA_DIR

'/Users/joeskvarna/Desktop/Thesis/Data'

In [38]:
city_dataset.to_csv(r'/Users/joeskvarna/Desktop/Thesis/Data/city_dataset_draft1.csv', index = False)

In [39]:
city_us

,Urban Cluster,Country,Population,Footprint (Mt CO2),1StdDev,Footprint/Cap (t CO2/cap),ClusterID,Global ranking
0,New Orleans,USA,596000,NaN,7.8,26.1 &plusmn;13.0,373,12
1,Detroit,USA,2770000,NaN,15.4,25.5 &plusmn;5.6,16079,13
...,...,...,...,...,...,...,...,...
107,Unknown city at lat/lon 33.5,USA,193000,NaN,1.3,9.6 &plusmn;6.9,39,425
108,Unknown city at lat/lon 27.3,USA,170000,NaN,0.8,9.2 &plusmn;5.0,550,471


In [40]:
#I will leave out 'Riverside-San Bernardino instead of combining with Riverside. This is a big area'

In [41]:
#right now I will elect to eliminate all rows that don't have a conservatism scale as this is the study I am going off.

In [42]:
city_dataset = city_dataset.loc[city_dataset['conservatism'].notna()]
city_dataset

,city,mayor_name,CO2_per_capita,conservatism,supports_paris
0,New Orleans,LaToya Cantrell,26.1 &plusmn;13.0,-0.51,True
1,Detroit,Mike Duggan,25.5 &plusmn;5.6,-0.73,True
...,...,...,...,...,...
106,Fresno,NaN,13.3 &plusmn;5.5,0.00,False
107,Las Vegas,NaN,12.3 &plusmn;4.8,-0.03,False


In [43]:
city_dataset.to_csv(r'/Users/joeskvarna/Desktop/Thesis/Data/city_dataset_draft2.csv', index = False)

In [44]:
city_dataset.head()

,city,mayor_name,CO2_per_capita,conservatism,supports_paris
0,New Orleans,LaToya Cantrell,26.1 &plusmn;13.0,-0.51,True
1,Detroit,Mike Duggan,25.5 &plusmn;5.6,-0.73,True
2,Cleveland,Frank Jackson,24.7 &plusmn;9.0,-0.42,True
3,St. Louis,Lyda Krewson,24.1 &plusmn;8.2,-0.52,True
4,Pittsburgh,William Peduto,24.0 &plusmn;10.4,-0.49,True


In [45]:
pd.options.mode.chained_assignment = None
city_dataset['CO2_per_cap'] = city_dataset['CO2_per_capita'].str.split(' &plusmn;').str[0]
city_dataset['plus_minus'] = city_dataset['CO2_per_capita'].str.split(' &plusmn;').str[1]
city_dataset

,city,mayor_name,CO2_per_capita,conservatism,supports_paris,CO2_per_cap,plus_minus
0,New Orleans,LaToya Cantrell,26.1 &plusmn;13.0,-0.51,True,26.1,13.0
1,Detroit,Mike Duggan,25.5 &plusmn;5.6,-0.73,True,25.5,5.6
...,...,...,...,...,...,...,...
106,Fresno,NaN,13.3 &plusmn;5.5,0.00,False,13.3,5.5
107,Las Vegas,NaN,12.3 &plusmn;4.8,-0.03,False,12.3,4.8


In [46]:
pd.options.mode.chained_assignment = 'warn'
city_dataset = city_dataset[['city', 'mayor_name', 'CO2_per_cap', 'plus_minus', 'conservatism', 'supports_paris']]
city_dataset

,city,mayor_name,CO2_per_cap,plus_minus,conservatism,supports_paris
0,New Orleans,LaToya Cantrell,26.1,13.0,-0.51,True
1,Detroit,Mike Duggan,25.5,5.6,-0.73,True
...,...,...,...,...,...,...
106,Fresno,NaN,13.3,5.5,0.00,False
107,Las Vegas,NaN,12.3,4.8,-0.03,False


In [47]:
city_dataset.to_csv(r'/Users/joeskvarna/Desktop/Thesis/Data/city_dataset_draft3.csv', index = False)

In [48]:
city_dataset['mayor_political_party'] = np.nan
city_dataset.head()

,city,mayor_name,CO2_per_cap,plus_minus,conservatism,supports_paris,mayor_political_party
0,New Orleans,LaToya Cantrell,26.1,13.0,-0.51,True,NaN
1,Detroit,Mike Duggan,25.5,5.6,-0.73,True,NaN
2,Cleveland,Frank Jackson,24.7,9.0,-0.42,True,NaN
3,St. Louis,Lyda Krewson,24.1,8.2,-0.52,True,NaN
4,Pittsburgh,William Peduto,24.0,10.4,-0.49,True,NaN


In [49]:
city_dataset['city'].duplicated(keep = False)

0      False
1      False
       ...  
106    False
107    False
Name: city, Length: 53, dtype: bool

In [50]:
city_dataset.loc[[66, 67]]

,city,mayor_name,CO2_per_cap,plus_minus,conservatism,supports_paris,mayor_political_party
66,Portland,Ethan Strimling,16.7,8.0,-0.59,True,NaN
67,Portland,Ted Wheeler,16.7,8.0,-0.59,True,NaN


In [51]:
if city_dataset['mayor_name'].loc[66] == 'Ethan Strimling':
    city_dataset.drop([66], inplace = True)
city_dataset

,city,mayor_name,CO2_per_cap,plus_minus,conservatism,supports_paris,mayor_political_party
0,New Orleans,LaToya Cantrell,26.1,13.0,-0.51,True,NaN
1,Detroit,Mike Duggan,25.5,5.6,-0.73,True,NaN
...,...,...,...,...,...,...,...
106,Fresno,NaN,13.3,5.5,0.00,False,NaN
107,Las Vegas,NaN,12.3,4.8,-0.03,False,NaN


In [52]:
#manually fill in Mayor name with a dict
city_dataset[['city', 'mayor_name']].loc[city_dataset['mayor_name'].isna()]

,city,mayor_name
9,Tulsa,NaN
16,Wichita,NaN
...,...,...
106,Fresno,NaN
107,Las Vegas,NaN


In [53]:
mayor_names = {'Tulsa': 'Dewey F. Bartlett Jr.', 'Wichita': 'Jeff Longwell', 'Oklahoma City': 'Mick Cornett', 'Omaha': 'Jean Stothert',
              'Colorado Springs': 'John Suthers', 'Jacksonville': 'Lenny Curry', 'Corpus Christi': 'Joe McComb', 'El Paso': 'Dee Margo',
              'Fresno': 'Lee Brand', 'Las Vegas': 'Carolyn Goodman'}

key_list = list(mayor_names.keys())
val_list = list(mayor_names.values())
mayor_df = DataFrame(key_list)  
mayor_df['mayor_name'] = val_list
mayor_df.rename(columns = {0: 'city'}, inplace = True)
mayor_df

,city,mayor_name
0,Tulsa,Dewey F. Bartlett Jr.
1,Wichita,Jeff Longwell
...,...,...
8,Fresno,Lee Brand
9,Las Vegas,Carolyn Goodman


In [54]:
city_dataset_merge = pd.merge(city_dataset, mayor_df, how='left', on = 'city')
city_dataset_merge

city_dataset_merge['mayor_name'] = city_dataset_merge['mayor_name_x'].fillna(city_dataset_merge['mayor_name_y'])
city_dataset_merge = city_dataset_merge.drop(['mayor_name_x', 'mayor_name_y'], axis = 1)
city_dataset = city_dataset_merge[['city', 'mayor_name','CO2_per_cap', 
                                   'plus_minus', 'conservatism', 'supports_paris',  'mayor_political_party']]
city_dataset

,city,mayor_name,CO2_per_cap,plus_minus,conservatism,supports_paris,mayor_political_party
0,New Orleans,LaToya Cantrell,26.1,13.0,-0.51,True,NaN
1,Detroit,Mike Duggan,25.5,5.6,-0.73,True,NaN
...,...,...,...,...,...,...,...
50,Fresno,Lee Brand,13.3,5.5,0.00,False,NaN
51,Las Vegas,Carolyn Goodman,12.3,4.8,-0.03,False,NaN


In [55]:
city_dataset1 = city_dataset.sort_values('mayor_name')
city_dataset1.to_csv(r'/Users/joeskvarna/Desktop/Thesis/Data/city_dataset_draft4.csv', index = False)

Final Step: Fill in mayor political party row

In [56]:
#After searching for possible sites to webscrape on, it will be easiest to fill in this column manually
#Here is the dataset with the last row filled in
city_dataset = pd.read_csv(path.join(DATA_DIR, 'city_dataset_reimport.csv'))
city_dataset

,city,mayor_name,CO2_per_cap,plus_minus,conservatism,supports_paris,mayor_political_party
0,Columbus,Andrew Ginther,19.8,8.5,-0.26,True,D
1,Baltimore,Bernard Young,20.2,9.1,-0.66,True,D
...,...,...,...,...,...,...,...
50,Toledo,Wade Kapszukiewicz,22.8,13.5,-0.17,True,D
51,Pittsburgh,William Peduto,24.0,10.4,-0.49,True,D
